# Section 2.1 - A Distributed Training Example

We will train a splitNN model that has been distributed to three different hosts. One host, Alice, is the data subject. Alice has the labelled data and will also be the custodian of the network start and end segments. Claire and Bob are worker hosts. They will feed the activation signals from the start of the chain forward until it reaches alices end layer. They will do the reverse with gradients in the backpropogation process. 

## Section 2.1.1 - Set up environmental variables

Here we will import our required libraries and initialise our model segments and data. We will need;

<img src="images/distributed.png" width="50%">

- A dummy distributed dataset
- 5 model segments
- 3 Virtual Workers

In [1]:
import torch
from torch import nn
from torch import optim
import syft as sy
import time
hook = sy.TorchHook(torch)

#from torchviz import make_dot, make_dot_from_trace
from torch.autograd import Variable

/Users/adamhall/anaconda3/envs/pysyft/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/Users/adamhall/anaconda3/envs/pysyft/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/Users/adamhall/anaconda3/envs/pysyft/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/Users/adamhall/anaconda3/envs/pysyft

In [2]:
@property
def location(self):
    m = self.__getitem__(0)
    w = m.weight[0]
    return w.location

nn.Sequential.location = location

In [3]:
# A Toy Dataset
x = torch.tensor([[0,0,0,0],[1,0,0,0],[0,1,0,0],[0,0,1,0],[1,1,0,0],[1,0,1,0],[0,1,1,0],[1,1,1,0],[0,0,0,1],[1,0,0,1],[0,1,0,1],[0,0,1,1],[1,1,0,1],[1,0,1,1],[0,1,1,1],[1,1,1,1.]])
y = torch.tensor([[0],[0],[0],[0],[0],[0],[0],[0],[1],[1],[1],[1],[1],[1],[1],[1.]])

torch.manual_seed(1)

# Define 5 chained models
models = [
    nn.Sequential(
        nn.Linear(4, 3),
        nn.Tanh()
    ),
    nn.Sequential(
        nn.Linear(3, 3),
        nn.Sigmoid()
    ),
    nn.Sequential(
        nn.Linear(3, 3),
        nn.Sigmoid()
    ),
    nn.Sequential(
        nn.Linear(3, 2),
        nn.Tanh()
    ),
    nn.Sequential(
        nn.Linear(2, 1),
        nn.Sigmoid()
    )
]

# create some workers
alice = sy.VirtualWorker(hook, id="alice")
bob = sy.VirtualWorker(hook, id="bob")
claire = sy.VirtualWorker(hook, id="claire")
workers = alice, bob, claire

The final predictions are shown above, we can compare this with the output of the same 'split' neural network

## Section 2.1.2 - Send Variables to Starting Locations

In this example, Alice is the worker with the data and labels. Bob and Claire are intermediary hosts in the chain. Alice has the start and end model segments. Bob and Claire have intermediary segments.

We send the models and data to their respective hosts and store the pointers in associative arrays; the Model Chain (MC) and the xy Chain (xyC). These contain the locations of the data, but no actual values. These are the only necessary parameters for coordinating this learning process. A summary of this is seen below

<img src="images/Parameters.png" width="50%">

In this experiment, the models and data are initialised locally and then distributed out.

In [4]:
# Send Model Segments to starting locations
model_locations = [alice, alice, bob, claire, alice]

for model, location in zip(models, model_locations):
    model.send(location)

# Create a remote copy of the dataset for each worker
datasets = [
    sy.BaseDataset(x.send(worker), y.send(worker))
    for worker in (alice, bob, claire)
]

In [5]:
def forward(models, x):

    inputs = []
    outputs = []
    
    # First: provide x as input
    inputs.append(x)
    outputs.append(models[0](x))    
    
    # Set the next input as the output of the previous model and move if necessary
    next_input = outputs[-1].clone()
    if next_input.location != models[1].location:
        next_input.move(models[1].location)
        
    for i in range(1, len(models)-1):
        inputs.append(next_input)
        outputs.append(models[i](next_input))
        next_input = outputs[-1].clone()
        
        if next_input.location != models[i+1].location:
            next_input.move(models[i+1].location)
 
    # Last: don't move the result to the next location
    inputs.append(next_input)
    outputs.append(models[len(models)-1](next_input))
    
    return inputs, outputs

In [6]:
inputs, outputs = forward(models, datasets[0].data)

for i in inputs:
    print(i.get())

tensor([[0., 0., 0., 0.],
        [1., 0., 0., 0.],
        [0., 1., 0., 0.],
        [0., 0., 1., 0.],
        [1., 1., 0., 0.],
        [1., 0., 1., 0.],
        [0., 1., 1., 0.],
        [1., 1., 1., 0.],
        [0., 0., 0., 1.],
        [1., 0., 0., 1.],
        [0., 1., 0., 1.],
        [0., 0., 1., 1.],
        [1., 1., 0., 1.],
        [1., 0., 1., 1.],
        [0., 1., 1., 1.],
        [1., 1., 1., 1.]])
tensor([[ 0.1806, -0.1924, -0.0364],
        [ 0.4138, -0.5821,  0.0330],
        [-0.0381,  0.1046, -0.0974],
        [ 0.0855, -0.2892,  0.1019],
        [ 0.2161, -0.3502, -0.0282],
        [ 0.3304, -0.6460,  0.1701],
        [-0.1342,  0.0021,  0.0410],
        [ 0.1220, -0.4370,  0.1101],
        [ 0.3947,  0.0595, -0.0118],
        [ 0.5882, -0.3895,  0.0577],
        [ 0.1941,  0.3447, -0.0729],
        [ 0.3098, -0.0433,  0.1262],
        [ 0.4254, -0.1109, -0.0035],
        [ 0.5212, -0.4731,  0.1939],
        [ 0.0993,  0.2510,  0.0656],
        [ 0.3428, -0.2110,  

In [7]:
def backward(models, optimizers, segment_inputs, segment_outputs, dataset):
    data, targets = dataset.data, dataset.targets
        
    # Destroy pre-existing gradient of final layer
    optimizers[len(optimizers)-1].zero_grad()
       
    loss = (((segment_outputs[-1] - targets)**2).sum())

    # Compute gradients
    loss.backward()
    
    # End layer sends the gradient of the activation signal back to the layer behind
    input_gradient = segment_inputs[-1].grad.clone()
    if input_gradient.location != models[len(models)-2].location:
        input_gradient.move(models[len(models)-2].location)
        
#     if segment_outputs[-2].location != models[len(models)-2].location:
#         segment_outputs[-2].move(models[len(models)-2].location)
    
    # End layer updates weights
    optimizers[-1].step()

    # Compute Intermediary Layers: repeat the same operations
    for iter in range(len(models)-1, 1, -1): 
        print("INTO FOR")
        optimizers[iter-1].zero_grad()
                
        intermediate_loss = torch.matmul(torch.t(segment_outputs[iter-1]), input_gradient).sum()
        intermediate_loss.backward()
        
        print(intermediate_loss.location)
        print(input_gradient.location)
        print(segment_inputs[iter-1].grad)
        print(segment_inputs[iter-1].location)


        input_gradient = segment_inputs[iter-1].grad.clone()
        if input_gradient.location != models[iter-2].location:
            input_gradient.move(models[iter-2].location)
            
#         if segment_outputs[iter-2].location != models[iter-2].location:
#             segment_outputs[iter-2].move(models[iter-2].location)

        
#         input_gradient = segment_inputs[iter-1].grad.clone().get().send(models[iter-2].location)
        optimizers[iter-1].step()
        print(iter)

        

    # Compute Final Layer, same but now input is the real input data
    optimizers[0].zero_grad()
    segment_output = segment_outputs[0]
    intermediate_loss = torch.matmul(torch.t(segment_output), input_gradient).sum()
    intermediate_loss.backward()
    optimizers[0].step()
        
    return segment_outputs[-1], loss

In [8]:
len(models)-1

4

## Section 2.1.5 - Run Training Logic

Now we will run the training process over 200 epochs for each data owner. Every 20 epochs we will print our progress. The front and end sections of the model will be swapped between data owners training each individual batch.

<img src="images/BatchFlow.png" width="40%">


In [9]:
def splitNN_train(models, xyChain):
    
    #   Variables for performance metrics
    start_time = time.time()
    epochs = 300
    lr = 0.2
    counter = 0
    
    # Create optimisers for each segment and link to their segment
    optimizers = [
        optim.SGD(params=model.parameters(),lr=lr)
        for model in models
    ]
    
    for i, local_worker in enumerate(workers):
        
        # Begin work on current data subject
        dataset = datasets[i]
        
        print('*', dataset.location.id, models[0].location.id)
        
        for epoch in range(epochs):
            # Forward propogate through network until final layer is reached
            segment_inputs, segment_outputs = forward(models, dataset.data)
            
            # Backward propogate
            predictions, loss = backward(models, optimizers, segment_inputs, segment_outputs, dataset)

            if epoch % 30 == 0:
                print(f"Epoch: {epoch}/{epochs} \tLoss: ", "{:.4f}\tRuntime: {:.2f}s".format(loss.get().data, time.time() - start_time))
        
        # If we are not at the end of the data owner chain send perimeter segments to next data owner
        if i < len(workers)-1:
            models[0].get().send(datasets[i+1].location)
            models[len(models)-1].get().send(datasets[i+1].location)      
            

            print("\nNEXT DATA OWNER\n")
            print("MODEL CHAIN LOCATIONS")
            for iter in range(len(models)):
                print(models[iter].location.id)  
            print("\n")
    
    # Send models back to researcher
    [model.get() for model in models]
    
    # Perform predictions with updates weights
    out = torch.tensor([[0,0,0,0],[1,0,0,0],[0,1,0,0],[0,0,1,0],[1,1,0,0],[1,0,1,0],[0,1,1,0],[1,1,1,0],[0,0,0,1],[1,0,0,1],[0,1,0,1],[0,0,1,1],[1,1,0,1],[1,0,1,1],[0,1,1,1],[1,1,1,1.]])
    for i in range(len(models)):
        out = models[i](out)
        
    print("\n\nFinal Predictions:", torch.t(out).data)
    

In [10]:
splitNN_train(models, datasets)

* alice alice
INTO FOR
<VirtualWorker id:claire #objects:9>
<VirtualWorker id:claire #objects:9>
(Wrapper)>[PointerTensor | me:77607064668 -> claire:70104102600]::grad
<VirtualWorker id:claire #objects:9>
4
INTO FOR
<VirtualWorker id:bob #objects:9>
<VirtualWorker id:bob #objects:9>
(Wrapper)>[PointerTensor | me:67038186143 -> bob:32974751714]::grad
<VirtualWorker id:bob #objects:9>
3
INTO FOR
<VirtualWorker id:alice #objects:20>
<VirtualWorker id:alice #objects:20>
None
<VirtualWorker id:alice #objects:20>


AttributeError: 'NoneType' object has no attribute 'clone'

# Notes

- I figured out why move() was breaking the forward prop function; it was trying to send to itself which caused an error.
- Gradient system breaks when not moving input tensors.
- Problems of 'knowledge of other models' in the training process can be solved by reversing the order from Claire to Alice. This removes the need for encrypting model segments.
- After I get to the bottom of the gradients, I will plug this into the MNIST and CIFAR datasets and for benchmarking. 